In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
%pylab inline
sb.set()

Populating the interactive namespace from numpy and matplotlib


In [ ]:
intakes = pd.read_csv('https://raw.githubusercontent.com/jwong002/sc1015project/main/Austin_Animal_Center_Intakes_20240327.csv')
intakes.columns = ['Animal ID', 'Name', 'DateTime', 'MonthYear', 'Found_Location',
       'Intake_Type', 'IntakeCondition', 'Animal_Type', 'Sex',
       'Age', 'Breed', 'Color']

outcomes = pd.read_csv('https://raw.githubusercontent.com/jwong002/sc1015project/main/Austin_Animal_Center_Outcomes_20240327.csv')
outcomes.columns = ['Animal ID', 'Name', 'DateTime', 'MonthYear', 'DOB',
       'Outcome_Type', 'Outcome_Subtype', 'Animal_Type',
       'Sex_upon_Outcome', 'Age_upon_Outcome', 'Breed', 'Color']

In [ ]:
# Sort the data of intakes and outcomes by DateTime
intakes.sort_values(by='DateTime', inplace=True)
outcomes.sort_values(by='DateTime', inplace=True)

# Identify duplicate IDs for intakes
duplicate_ids_intakes = intakes['Animal ID'].duplicated(keep=False)

# Generate unique suffixes for each duplicate ID based on their occurrence order
suffixes_intakes = intakes[duplicate_ids_intakes].groupby('Animal ID').cumcount().astype(str)

# Map suffixes to letters (a, b, c, ...)
suffixes_intakes = suffixes_intakes.map(lambda x: chr(ord('a') + int(x)))

# Add the suffixes to the duplicate IDs
intakes.loc[duplicate_ids_intakes, 'Animal ID'] += suffixes_intakes

# Identify duplicate IDs for outcomes
duplicate_ids_outcomes = outcomes['Animal ID'].duplicated(keep=False)

# Generate unique suffixes for each duplicate ID based on their occurrence order
suffixes_outcomes = outcomes[duplicate_ids_outcomes].groupby('Animal ID').cumcount().astype(str)

# Map suffixes to letters (a, b, c, ...)
suffixes_outcomes = suffixes_outcomes.map(lambda x: chr(ord('a') + int(x)))

# Add the suffixes to the duplicate IDs
outcomes.loc[duplicate_ids_outcomes, 'Animal ID'] += suffixes_outcomes

In [ ]:
Records = intakes.merge(outcomes, how='inner', left_on = 'Animal ID', right_on = 'Animal ID', suffixes=('_intake', '_outcome'))
Records.shape

(159792, 23)

In [ ]:
Records['MonthYear_intake'] = pd.to_datetime(Records['MonthYear_intake'])
Records['MonthYear_outcome'] = pd.to_datetime(Records['MonthYear_outcome'])

<ipython-input-45-7a6ac1e59270>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  Records['MonthYear_intake'] = pd.to_datetime(Records['MonthYear_intake'])
<ipython-input-45-7a6ac1e59270>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  Records['MonthYear_outcome'] = pd.to_datetime(Records['MonthYear_outcome'])


In [ ]:
Records.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159792 entries, 0 to 159791
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Animal ID            159792 non-null  object        
 1   Name_intake          113782 non-null  object        
 2   DateTime_intake      159792 non-null  object        
 3   MonthYear_intake     159792 non-null  datetime64[ns]
 4   Found_Location       159792 non-null  object        
 5   Intake_Type          159792 non-null  object        
 6   IntakeCondition      159792 non-null  object        
 7   Animal_Type_intake   159792 non-null  object        
 8   Sex                  159790 non-null  object        
 9   Age                  159791 non-null  object        
 10  Breed_intake         159792 non-null  object        
 11  Color_intake         159792 non-null  object        
 12  Name_outcome         113782 non-null  object        
 13  DateTime_outco

In [ ]:
Records.head()

,Animal ID,Name_intake,DateTime_intake,MonthYear_intake,Found_Location,Intake_Type,IntakeCondition,Animal_Type_intake,Sex,Age,...,DateTime_outcome,MonthYear_outcome,DOB,Outcome_Type,Outcome_Subtype,Animal_Type_outcome,Sex_upon_Outcome,Age_upon_Outcome,Breed_outcome,Color_outcome
0,A670057,Brownie,01/01/2014 01:05:00 PM,2014-01-01,S Lamar And Ben White in Austin (TX),Stray,Normal,Dog,Intact Male,2 years,...,01/05/2014 04:50:00 PM,2014-01-01,01/01/2012,Return to Owner,NaN,Dog,Neutered Male,2 years,Chow Chow Mix,Red
1,A670059,NaN,01/01/2014 01:31:00 PM,2014-01-01,11402 Robert Wooding in Austin (TX),Stray,Normal,Dog,Spayed Female,1 year,...,01/05/2014 02:37:00 PM,2014-01-01,01/01/2013,Adoption,NaN,Dog,Spayed Female,1 year,West Highland,White
2,A670061,Koda,01/01/2014 01:33:00 PM,2014-01-01,Austin (TX),Owner Surrender,Normal,Dog,Intact Male,2 years,...,01/24/2014 01:41:00 PM,2014-01-01,01/01/2012,Adoption,NaN,Dog,Neutered Male,2 years,Chow Chow Mix,Red
3,A670064,NaN,01/01/2014 01:57:00 PM,2014-01-01,Gunter St And Gonzalez in Austin (TX),Stray,Normal,Dog,Intact Female,3 months,...,01/03/2014 04:33:00 PM,2014-01-01,09/16/2013,Died,In Kennel,Dog,Intact Female,3 months,Pit Bull/Pit Bull,Red
4,A670065,Muneca,01/01/2014 02:11:00 PM,2014-01-01,Applewood Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,7 years,...,01/02/2014 11:52:00 AM,2014-01-01,01/02/2007,Return to Owner,NaN,Dog,Spayed Female,7 years,Australian Shepherd/Chow Chow,Red/White


# **Fixing records that don't match and dropping irrelevant columns**

# **Estimating the Relevance of Date of Birth Information**

In [ ]:
# DOB appears to be irrelevant because every animal has a DOB even though not all animals were born in the shelter

null_dobs = Records['DOB'].isnull().sum()
print("Null DOBs:", null_dobs)

Null DOBs: 0


# **Removing Redundant Columns**

In [ ]:
# Animal Type, Breed and Color are all consistent across intake and outake
columns_to_delete = ['DOB', 'Animal_Type_outcome', 'Breed_outcome', 'Color_outcome', 'Found_Location']
for col in columns_to_delete:
    if col in Records.columns:
        del Records[col]

# **Displaying the updated column structure of the data**

In [ ]:
print ("COLUMNS: ", Records.columns)

COLUMNS:  Index(['Animal ID', 'Name_intake', 'DateTime_intake', 'MonthYear_intake',
       'Intake_Type', 'IntakeCondition', 'Animal_Type_intake', 'Sex', 'Age',
       'Breed_intake', 'Color_intake', 'Name_outcome', 'DateTime_outcome',
       'MonthYear_outcome', 'Outcome_Type', 'Outcome_Subtype',
       'Sex_upon_Outcome', 'Age_upon_Outcome'],
      dtype='object')


# **Estimating missing values**

In [ ]:
Records.isnull().sum()

Animal ID                 0
Name_intake           46010
DateTime_intake           0
MonthYear_intake          0
Intake_Type               0
IntakeCondition           0
Animal_Type_intake        0
Sex                       2
Age                       1
Breed_intake              0
Color_intake              0
Name_outcome          46010
DateTime_outcome          0
MonthYear_outcome         0
Outcome_Type             35
Outcome_Subtype       86530
Sex_upon_Outcome          2
Age_upon_Outcome          2
dtype: int64

# **Filling in missing data values**

In [ ]:
Records["Name_intake"].fillna(value = "NoName", inplace = True)
Records["Name_outcome"].fillna(value = "NoName", inplace = True)
Records["Outcome_Type"].fillna(value = "Unknown", inplace = True)
Records["Outcome_Subtype"].fillna(value = "Unknown", inplace = True)
Records["Sex"].fillna(value = "Unknown", inplace = True)
Records["Age"].fillna(value = "Unknown", inplace = True)
Records["Sex_upon_Outcome"].fillna(value = "Unknown", inplace = True)
Records["Age_upon_Outcome"].fillna(value = "Unknown", inplace = True)
Records.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159792 entries, 0 to 159791
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Animal ID           159792 non-null  object        
 1   Name_intake         159792 non-null  object        
 2   DateTime_intake     159792 non-null  object        
 3   MonthYear_intake    159792 non-null  datetime64[ns]
 4   Intake_Type         159792 non-null  object        
 5   IntakeCondition     159792 non-null  object        
 6   Animal_Type_intake  159792 non-null  object        
 7   Sex                 159792 non-null  object        
 8   Age                 159792 non-null  object        
 9   Breed_intake        159792 non-null  object        
 10  Color_intake        159792 non-null  object        
 11  Name_outcome        159792 non-null  object        
 12  DateTime_outcome    159792 non-null  object        
 13  MonthYear_outcome   159792 no

# **Feature Extraction**

# **Derive the statuses "Spayed" and "Intact" to determine if there were any alterations in these conditions throughout the shelter stay, as well as ascertain the gender of each animal.**

In [ ]:
def get_gender(val):
    try:
        val_list = val.split(' ')
        return val_list[1]
    except:
        return np.nan
def get_fixed(val):
    try:
        val_list = val.split(' ')
        return val_list[0]
    except:
        return np.nan

Records['Gender'] = Records['Sex'].apply(get_gender)
Records['fixed_intake'] = Records['Sex'].apply(get_fixed)
Records['fixed_outcome'] = Records['Sex_upon_Outcome'].apply(get_fixed)
Records["Gender"].fillna(value = "Unknown", inplace = True)

Records['fixed_changed'] = 0
mask = (Records['fixed_intake'] == 'Intact') & (Records['fixed_outcome'] != 'Intact')
Records.loc[mask, 'fixed_changed'] = 1
mask = (Records['fixed_intake'] != 'Intact') & (Records['fixed_outcome'] == 'Intact')
Records.loc[mask, 'fixed_changed'] = -1

value_descriptions = {
0: "No change in fixed status (0):",
1: "Changed from intact at intake to not intact at outcome (1):",
-1: "Changed from not intact at intake to intact at outcome (-1):"
}

fixed_changed_counts = Records['fixed_changed'].value_counts().rename(index=value_descriptions)
print(fixed_changed_counts)


fixed_changed
No change in fixed status (0):                                  95185
Changed from intact at intake to not intact at outcome (1):     64563
Changed from not intact at intake to intact at outcome (-1):       44
Name: count, dtype: int64


# **Categorizing age into ranges**

In [ ]:
import re

def get_age_bucket(val):
    if re.search("day", val):
        return "Less than 1 week"
    elif re.search("week", val):
        return "1-6 weeks"
    elif re.search("[1,2,3,4,5,6] month", val):
        return "1-6 months"
    elif re.search("[7,8,9,10,11,12] month", val):
        return "7-12 months"
    elif re.search("[1,2,3] year", val):
        return "1-3 years"
    elif re.search("[4,5,6] year", val):
        return "4-6 years"
    elif re.search("year", val):
        return "7+ years"
    else:
        return "other"

Records['Age_Bucket'] = Records['Age'].apply(get_age_bucket)
Records['Age_Bucket'].value_counts()

Age_Bucket
1-3 years           63660
1-6 months          42958
1-6 weeks           16682
4-6 years           14526
7+ years            10571
7-12 months          7898
Less than 1 week     3496
other                   1
Name: count, dtype: int64

# **Evaluate the duration of an animal's stay by calculating the gap between arrival and departure dates, followed by grouping these durations into defined intervals.**

In [ ]:
import pandas as pd
import numpy as np
import re

# Convert columns to datetime if they are not already
Records['DateTime_intake'] = pd.to_datetime(Records['DateTime_intake'])
Records['DateTime_outcome'] = pd.to_datetime(Records['DateTime_outcome'])

# Calculate the length of stay in the shelter as a new column
Records['DateTime_length'] = Records['DateTime_outcome'] - Records['DateTime_intake']

# Remove negative values under DateTime_length column
Records = Records[Records['DateTime_length'] >= pd.Timedelta(days=0)]

# Calculate quantiles for filtering outliers
q_low = Records['DateTime_length'].quantile(0.01)
q_hi = Records['DateTime_length'].quantile(0.99)

# Filter records between the lower and upper quantiles to remove outliers
Records = Records[(Records['DateTime_length'] >= q_low) & (Records['DateTime_length'] <= q_hi)]

def get_days_length(val):
    try:
        days = val.days  # Extract the number of days from the timedelta
        if days <= 7:
            return "0-7 days"
        elif days <= 21:
            return "1-3 weeks"
        elif days <= 42:
            return "3-6 weeks"
        elif days <= 84:
            return "7-12 weeks"
        elif days <= 168:
            return "12 weeks - 6 months"
        elif days <= 365:
            return "6-12 months"
        elif days <= 730:
            return "1-2 years"
        else:
            return "2+ years"
    except:
        return np.nan

Records['Days_length'] = Records['DateTime_length'].apply(get_days_length)
import pandas as pd

# Convert 'DateTime_length' to a 'Timedelta' object if it's not already
Records['DateTime_length'] = pd.to_timedelta(Records['DateTime_length'])

# Extract just the days component of 'DateTime_length'
Records['Days_length2'] = Records['DateTime_length'].dt.days

# Now, 'Days_length' will have only the number of days, without hours, minutes, and seconds

print(Records['Days_length'].value_counts())
print(Records['Days_length2'].head())

<ipython-input-57-71f9c1989811>:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  Records['DateTime_intake'] = pd.to_datetime(Records['DateTime_intake'])
<ipython-input-57-71f9c1989811>:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  Records['DateTime_outcome'] = pd.to_datetime(Records['DateTime_outcome'])


Days_length
0-7 days               90955
1-3 weeks              28503
3-6 weeks              14796
7-12 weeks             12967
12 weeks - 6 months     4929
6-12 months             1708
Name: count, dtype: int64
0     4
1     4
2    23
3     2
4     0
Name: Days_length2, dtype: int64


# **Exporting the cleaned DataFrame to a new CSV file**

In [ ]:
cleaned_data_path = 'train_cleaned.csv'
Records.to_csv(cleaned_data_path, index=False)

In [ ]:
from google.colab import files
files.download('train_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>